In [ ]:
import numpy as np
import warnings
import os
    
def scan_ideal(neur_vol = None,PSF_struct = None,neur_act = None,scan_params = None,varargin = None): 
    # mov = scan_ideal(neur_vol, PSF, neur_act, scan_params, varargin)
    
    # Scan a volume and create ideal scanned components.
# The inputs to this function are:
#   neur_vol     - A struct that contains the output of the volume
#                  generating code.
#   PSF          - A struct representing the point-spread function of the
#                  microscopy setup
#   neur_act     - Struct containing the simulated fluorescence activity
#                  for each of the K neurons for all nt time-steps
#    .soma       - Kx(nt) matrix where each row is the activity of each
#                  cell at the soma
#    .dend       - Kx(nt) matrix where each row is the activity of each
#                  cell at the dendrites [can be optional]
#    .bg         - N_bgx(nt) matrix where each row is the activity of each
#                  background/neuropil component [can be optional]
#   scan_params  - Struct contaning the scanning parameters. Includes
#    .scan_avg   - Sampling rate of the scanning in terms of how many
#                  granular pixels to scan into one pixel (default = 2)
#    .motion     - True/false option for whether or not to simulate
#                  motion while performing scanning (default = true)
#    .scan_buff  - Number of granular pixels to keep as a buffer from the
#                  edge of the volume (default = 10)
#    .verbose    - Level of verbosity in the output during the volume
#                  generation. Can be 0,1,2. 0 = no text updates, 1 =
#                  some text outputs. 2 = detailed text outputs (default
#                  = 1)
#   noise_params - Struct containing parameters for the noise model
#    .mu         - Mean measurement increase per photon (default = 0.5)
#    .mu0        - Electronics offset (default = 0)
#    .sigma      - Variance increase per photon (default = 0.3)
#    .sigma0     - Electronics base noise variance (default = 0.1)
    
    # The outputs of this function is:
#   mov         - A 3D array where each 2D slice is one frame in the video
#                 recroding of the simulated microscopy data
    
    # 2016 - Adam Charles and Alex Song
    
    ###########################################################################
## Parse inputs
    
    if not isfield(PSF_struct,'psf') :
        raise Exception('Must provide PSF to scan!')
    else:
        PSF = PSF_struct.psf
    
    if not isfield(PSF_struct,'mask') :
        t_mask = []
    else:
        t_mask = PSF_struct.mask
        t_thresh = 1e-05
        t_mask[t_mask < t_thresh] = t_thresh
    
    if isfield(PSF_struct,'colmask'):
        t_mask = np.multiply(t_mask,PSF_struct.colmask)
    
    if not isfield(PSF_struct,'g_blur') :
        g_blur = []
    else:
        g_blur = PSF_struct.blur
    
    if len(varargin) > 4:
        noise_params = varargin[0]
    else:
        noise_params = struct
    
    scan_params = check_scan_params(scan_params)
    
    noise_params = check_noise_params(noise_params)
    
    if (int(np.floor(scan_params('sfrac'))) != scan_params('sfrac')):
        warnings.warn('Binning parameter is not an integer - this causes local interpolation of pixel positions')
    
    if not isstruct(neur_act) :
        if np.shape(neur_act[2]) == 1:
            neur_act = struct('soma',neur_act)
        else:
            if np.shape(neur_act[2]) == 2:
                neur_act = struct('soma',neur_act(:,:,1),'dend',neur_act[:,:,2])
    
    if (not isfield(neur_act,'dend') ) or (len(neur_act.dend)==0):
        neur_act.dend = neur_act.soma
    
    if (not isfield(neur_act,'bg') ) or (len(neur_act.bg)==0):
        neur_act.bg = np.zeros((1,neur_act.shape[2-1],'single'))
    
    if (isfield(scan_params,'nuc_label')) and (scan_params('nuc_label') >= 1):
        neur_act.nuc = neur_act.soma
        neur_act.soma = 0 * neur_act.soma
        neur_act.dend = 0 * neur_act.dend
        neur_act.bg = 0 * neur_act.bg
    
    if (not isfield(scan_params,'fsimCleanPath') ):
        scan_params.fsimCleanPath = []
    
    if (not isfield(scan_params,'saveBlocksize') ) or (len(scan_params('saveBlocksize'))==0):
        scan_params.saveBlocksize = 1000
    
    if (not isfield(scan_params,'movout') ) or (len(scan_params('movout'))==0):
        scan_params.movout = 1
    
    ###########################################################################
## Initialize data
    
    n0 = np.amin(neur_act.soma,[],2)
    
    n1 = np.amin(neur_act.dend,[],2)
    
    n2 = np.amin(neur_act.bg,[],2)
    
    gIdxs = (n0 + n1 + n2) > 0
    neur_base.soma = np.zeros((np.shape(neur_act.soma[0]),sum(gIdxs),'single'))
    
    neur_base.dend = np.zeros((np.shape(neur_act.dend[0],sum(gIdxs),'single'))
    
    neur_base.bg = np.zeros((np.shape(neur_act.bg[0],sum(gIdxs),'single'))
    
    neur_base('soma')[sub2ind[neur_base.soma.shape,np.transpose[find[gIdxs]],np.arange[1,sum[gIdxs]+1]]] = n0(gIdxs)
    neur_base('dend')[sub2ind[neur_base.dend.shape,np.transpose[find[gIdxs]],np.arange[1,sum[gIdxs]+1]]] = n1(gIdxs)
    neur_base('bg')[sub2ind[neur_base.bg.shape,np.transpose[find[gIdxs]],np.arange[1,sum[gIdxs]+1]]] = n2(gIdxs)
    neur_act = neur_base
    clear('neur_base')
    if (scan_params('scan_avg') > 1):
        warnings.warn('Warning: Scan averages greater than 1 not supported for more than 1 offset')
    
    scan_avg = scan_params('scan_avg')
    
    sfrac = scan_params('sfrac')
    
    if (not isfield(scan_params,'vol_sz') ) or (len(scan_params('vol_sz')==0):
        N1 = neur_vol.neur_vol.shape[1-1]
        N2 = neur_vol.neur_vol.shape[2-1]
        N3 = neur_vol.neur_vol.shape[3-1]
    else:
        N1 = scan_params.vol_sz(1)
        N2 = scan_params.vol_sz(2)
        N3 = scan_params.vol_sz(3)
    
    Nt = neur_act.soma.shape[2-1]
    
    if not isstruct(PSF) :
        Np1,Np2,Np3 = PSF.shape
    else:
        if isfield(PSF,'left'):
            Np1,Np2,Np3 = PSF.left.shape
        else:
            raise Exception('Unknown input configuration!')
    
    ###########################################################################
## Perform some error checking
    
    if (N1 < Np1) or (N2 < Np2):
        raise Exception('PSF extent is bigger than the volume!')
    
    if (N3 < Np3):
        raise Exception('PSF depth is larger than the volume depth!')
    
    ###########################################################################
## Initialize movie and temporary volume
    
    TMPvol = np.zeros(N1,N2,N3,'single')
    
    if (not isfield(scan_params,'zoffset') ) or (len(scan_params('zoffset'))==0):
        z_base = int(np.floor(0.5 * (N3 - Np3)))
    else:
        z_base = int(np.floor(0.5 * (N3 - Np3))) + scan_params('zoffset')
    
    ############### TO DO: MAKE INTO A FIELD
    zmaxdiff = 2
    ###############
    
    if (isfield(PSF_struct,'psfT')) and (not len(PSF_struct.psFT)==0 ):
        psfT = PSF_struct.psfT
        psfB = PSF_struct.psfB
        psfT.mask = psfT.mask / mean(psfT.mask)
        psfB.mask = psfB.mask / mean(psfB.mask)
        psfT.convmask = psfT.convmask / sum(psfT.convmask)
        psfB.convmask = psfB.convmask / sum(psfB.convmask)
        psfT.freq_psf = psf_fft(TMPvol.shape,psfT.convmask)
        psfB.freq_psf = psf_fft(TMPvol.shape,psfB.convmask)
    else:
        psfT = []
        psfB = []
    
    freq_psf = psf_fft(TMPvol.shape,PSF,scan_avg)
    
    sigscale = noise_params.sigscale
    
    somaVol = cell(neur_vol.gp_vals.shape[1-1],2)
    
    dendVol = cell(neur_vol.gp_vals.shape[1-1],2)
    
    for i in np.arange(1,neur_vol.gp_vals.shape[1-1]+1).reshape(-1):
        somaVol[i,1] = neur_vol.gp_vals[i,1](neur_vol.gp_vals[i,3])
        dendVol[i,1] = neur_vol.gp_vals[i,1](not neur_vol.gp_vals[i,3] )
        if (len(t_mask)==0):
            somaVol[i,2] = neur_vol.gp_vals[i,2](neur_vol.gp_vals[i,3])
            dendVol[i,2] = neur_vol.gp_vals[i,2](not neur_vol.gp_vals[i,3] )
        else:
            somaVol[i,2] = np.multiply(neur_vol.gp_vals[i,2](neur_vol.gp_vals[i,3]),t_mask(np.amax(1,np.mod(somaVol[i,1],N1 * N2))))
            dendVol[i,2] = np.multiply(neur_vol.gp_vals[i,2](not neur_vol.gp_vals[i,3] ),t_mask(np.amax(1,np.mod(dendVol[i,1],N1 * N2))))
    
    if (not len(neur_vol.bg_proc)==0 ):
        if (len(t_mask)==0):
            axonVol = neur_vol.bg_proc
        else:
            axonVol = cell(neur_vol.bg_proc.shape)
            for i in np.arange(1,neur_vol.bg_proc.shape[1-1]+1).reshape(-1):
                axonVol[i,1] = neur_vol.bg_proc[i,1]
                axonVol[i,2] = np.multiply(neur_vol.bg_proc[i,2],t_mask(np.amax(1,np.mod(axonVol[i,1],N1 * N2))))
    else:
        axonVol = []
    
    if (isfield(scan_params,'nuc_label')) and (scan_params.nuc_label >= 1):
        if (len(t_mask)==0):
            nucVol = neur_vol.gp_nuc
        else:
            nucVol = cell(neur_vol.gp_nuc.shape)
            for i in np.arange(1,neur_vol.gp_nuc.shape[1-1]+1).reshape(-1):
                nucVol[i,1] = neur_vol.gp_nuc[i,1]
                nucVol[i,2] = t_mask(np.amax(1,np.mod(nucVol[i,1],N1 * N2)))
    
    ###########################################################################
## Setup scanning volume
    
    cutoff = 0.01
    soma_act = single(neur_act.soma)
    soma_min = np.amin(soma_act,[],2)
    soma_act = np.subtract(soma_act,soma_min)
    soma_act[soma_act < cutoff] = 0
    dend_act = single(neur_act.dend)
    dend_min = np.amin(dend_act,[],2)
    dend_act = np.subtract(dend_act,dend_min)
    dend_act[dend_act < cutoff] = 0
    bg_act = single(neur_act.bg)
    bg_min = np.amin(bg_act,[],2)
    bg_act = np.subtract(minus,bg_act,bg_min)
    bg_act[bg_act < cutoff] = 0
    ###########################################################################
## Iteratively scan the volume
    
    if (not len(scan_params.fsimCleanPath)==0 ):
        tifLinkFsimClean = cell(2 * zmaxdiff + 1,1)
        tagFsimClean = cell(2 * zmaxdiff + 1,1)
        fsimCleanPath = cell(2 * zmaxdiff + 1,1)
        for jj in np.arange(1,(2 * zmaxdiff + 1)+1).reshape(-1):
            path,name,__ = os.path.split(scan_params.fsimCleanPath)[0],os.path.splitext(os.path.split(scan_params.fsimCleanPath)[1])[0],os.path.splitext(os.path.split(scan_params.fsimCleanPath)[1])[1]
            fsimCleanPath[jj] = fullfile(path,sprintf(np.array([name,'_%02d.tif']),jj))
            tifLinkFsimClean[jj],tagFsimClean[jj] = tifinitialize(fsimCleanPath[jj],int(np.floor(np.array([(N1) / sfrac,(N2) / sfrac]))))
    
    if scan_params.verbose == 1:
        print('Scanning...' % ())
    else:
        if scan_params('verbose') > 1:
            print('Scanning...\n' % ())
    
    if not isstruct(PSF) :
        for kk in np.arange(1,Nt+1).reshape(-1):
            if scan_params.verbose > 1:
                tic
            clear('TMPvol')
            TMPvol = np.zeros(N1,N2,N3,'single')
            for ll in np.arange(1,soma_act.shape[1-1]+1).reshape(-1):
                if (soma_act(ll,kk) > 0) and (not len(somaVol[ll,2])==0 ):
                    array_SubSubTest(TMPvol,somaVol[ll,1],somaVol[ll,2],soma_act(ll,kk))
            if (isfield(scan_params,'nuc_label')) and (scan_params.nuc_label >= 1):
                for ll in np.arange(1,neur_vol.gp_nuc.shape[1-1]+1).reshape(-1):
                    if (nuc_act(ll,kk) > 0) and (not len(nucVol[ll,2])==0 ):
                        array_SubSubTest(TMPvol,nucVol[ll,1],nucVol[ll,2],nuc_act(ll,kk))
            for ll in np.arange(1,dend_act.shape[1-1]+1).reshape(-1):
                if (dend_act(ll,kk) > 0) and (not len(dendVol[ll,1])==0 ):
                    array_SubSubTest(TMPvol,dendVol[ll,1],dendVol[ll,2],dend_act(ll,kk))
            for ll in np.arange(1,bg_act.shape[1-1]+1).reshape(-1):
                if (not len(axonVol[ll,1])==0  and bg_act(ll,kk) > 0):
                    array_SubModTest(TMPvol,axonVol[ll,1],axonVol[ll,2],bg_act(ll,kk))
            z_loc = int(np.floor(0.5 * (N3 - Np3)))
            clean_img_all = (sigscale / (2 * sfrac ** 2)) * single_scan_stack(TMPvol,PSF.shape,freq_psf,scan_avg,True,np.arange(z_loc,(z_loc + Np3 - 1)+1),np.arange(- zmaxdiff,zmaxdiff+1))
            for jj in np.arange(1,(2 * zmaxdiff + 1)+1).reshape(-1):
                z_loc = jj - zmaxdiff - 1 + int(np.floor(0.5 * (N3 - Np3)))
                clean_img = clean_img_all(:,:,jj)
                if (not len(g_blur)==0 ):
                    clean_img = clean_img + conv2(clean_img,g_blur,'same')
                if (not len(psfT)==0 ):
                    if (not len(t_mask)==0 ):
                        top_mask = 1.0 / t_mask
                        bot_mask = 1.0 / t_mask
                    else:
                        top_mask = np.ones((clean_img.shape[1-1],clean_img.shape[2-1],'single'))
                        bot_mask = np.ones((clean_img.shape[1-1],clean_img.shape[2-1],'single'))
                    if (isfield(psfT,'mask')):
                        top_mask = np.multiply(top_mask,psfT.mask)
                        bot_mask = np.multiply(bot_mask,psfB.mask)
                    if (len(np.arange(1,z_loc - 1+1))==0):
                        top_img = blurredBackComp2(TMPvol,np.arange(1,TMPvol.shape[3-1]+1),psfT.freq_psf,psfT.weight,top_mask,1,[])
                    else:
                        top_img = blurredBackComp2(TMPvol,np.arange(1,z_loc - 1+1),psfT.freq_psf,psfT.weight,top_mask,1,psfT.psfZ)
                    if (len(np.arange(z_loc + Np3,TMPvol.shape[3-1]+1))==0):
                        bot_img = blurredBackComp2(TMPvol,np.arange(1,TMPvol.shape[3-1]+1),psfB.freq_psf,psfB.weight,bot_mask,1,[])
                    else:
                        bot_img = blurredBackComp2(TMPvol,np.arange(z_loc + Np3,TMPvol.shape[3-1]+1),psfB.freq_psf,psfB.weight,bot_mask,1,psfB.psfZ)
                    top_img = top_img * (sigscale / (sfrac ** 2))
                    bot_img = bot_img * (sigscale / (sfrac ** 2))
                    clean_img = clean_img + top_img + bot_img
                if (int(np.floor(sfrac)) == sfrac):
                    clean_img = conv2(clean_img,np.ones((sfrac,sfrac)),'same')
                    clean_img = clean_img(np.arange(1,end()+sfrac,sfrac),np.arange(1,end()+sfrac,sfrac))
                else:
                    clean_img = sfrac * sfrac * imresize(clean_img,1 / sfrac,'bilinear')
                if (not len(scan_params('fsimCleanPath')==0 ):
                    tifLinkFsimClean[jj] = tifappend(tifLinkFsimClean[jj],clean_img,tagFsimClean[jj],scan_params.saveBlocksize,kk,fsimCleanPath[jj])
            if scan_params.verbose == 1:
                print('.' % ())
            else:
                if scan_params('verbose') > 1:
                    Ttmp = toc
                    print('Scanned frame %d (%f s)\n' % (kk,Ttmp))
    else:
        if isfield(PSF,'left'):
            raise Exception('Supported PSF configuration!')
        else:
            raise Exception('Unknown input configuration!')
    
    if (not len(scan_params('fsimCleanPath')==0 ):
        for jj in np.arange(1,(2 * zmaxdiff + 1)+1).reshape(-1):
            tifLinkFsimClean[jj].close()
    
    if scan_params('verbose') >= 1:
        print('done.\n' % ())
    
    return